# Set up

In [1]:
!python -m spacy download en_core_web_md
#!python -m spacy download en_core_web_lg

     |████████████████████████████████| 96.4 MB 1.2 MB/s 
  Created wheel for en-core-web-md: filename=en_core_web_md-2.2.5-py3-none-any.whl size=98051302 sha256=5a89b66a6ced29f2ad9a87fec1315aa1da4ce65ddcc3395532f2dd72a9bb7102
  Stored in directory: /tmp/pip-ephem-wheel-cache-1_t077eo/wheels/69/c5/b8/4f1c029d89238734311b3269762ab2ee325a42da2ce8edb997
Successfully built en-core-web-md
✔ Download and installation successful
You can now load the model via spacy.load('en_core_web_md')


In [2]:
!pip install polyglot
!pip install pyicu
!pip install pycld2
!pip install morfessor
!pip install metaphone

     |████████████████████████████████| 126 kB 7.6 MB/s 
  Created wheel for polyglot: filename=polyglot-16.7.4-py2.py3-none-any.whl size=52578 sha256=c888828ff14a3296c6a1581a183480b589bbc5b627030dcb32bb6e1507419109
  Stored in directory: /root/.cache/pip/wheels/09/bc/67/75c9de8e9726460bc0b101ad225ad025cb8ce9e0759beb9d52
Successfully built polyglot
     |████████████████████████████████| 299 kB 6.9 MB/s 
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
  Created wheel for pyicu: filename=PyICU-2.8-cp37-cp37m-linux_x86_64.whl size=1377122 sha256=790900783de858bdfe19a2e40bab993184534ac78fbc9a975d40e1c74f770f1f
  Stored in directory: /root/.cache/pip/wheels/14/bd/45/aeddc643bd0637c14fa27bffaee5b411cdc323f8bec76ad15e
Successfully built pyicu
     |████████████████████████████████| 41.4 MB 39 kB/s 
  Created wheel for pycld2: filename=pycld2-0.41-cp37-cp37m-linux_x86_64.whl size=9834240 sha256=25739c57eed1d6d675be8

In [3]:
import pandas as pd
import numpy as np
import os
import math
from google.colab import drive
import polyglot
import spacy
import nltk
import re

from nltk.tokenize import TweetTokenizer
from polyglot.text import Text, Word

import spacy
from spacy import displacy
from collections import Counter
import en_core_web_sm
import en_core_web_md
#import en_core_web_lg
from dateutil import parser
from sklearn import metrics
from sklearn.metrics import f1_score, roc_auc_score
from sklearn.model_selection import train_test_split
from sklearn import svm
from sklearn.svm import SVC
from sklearn.decomposition import PCA
from sklearn.metrics import confusion_matrix

from sklearn.metrics import accuracy_score, recall_score,precision_score,  classification_report
from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import Pipeline, FeatureUnion
from collections import Counter

In [4]:
drive.mount('/content/drive', force_remount= True)
os.chdir('/content/drive/MyDrive/Asesoftware/Plan semilla/NLP Challenge') 
#os.chdir("/content/drive/MyDrive/NLP Challenge") #Thomas
#os.chdir('/content/drive/MyDrive/ASESOFTWARE/NLP/NLP Challenge') #Sebastian
os.listdir("./")

Mounted at /content/drive


['data',
 'Thomas Edition IR Classiffication.ipynb',
 'results',
 'NN.ipynb',
 'IR Classiffication.ipynb',
 'Logistic Regression FTW.ipynb']

In [5]:
DATA_PATH = './data/'
#RESULTS_PATH = './results/'

In [6]:
train = pd.read_csv(DATA_PATH+'train.csv', encoding='utf-8')
test = pd.read_csv(DATA_PATH+'test.csv')
train.shape, test.shape

((7613, 5), (3263, 4))

In [7]:
test

,id,keyword,location,text
0,0,NaN,NaN,Just happened a terrible car crash
1,2,NaN,NaN,"Heard about #earthquake is different cities, s..."
2,3,NaN,NaN,"there is a forest fire at spot pond, geese are..."
3,9,NaN,NaN,Apocalypse lighting. #Spokane #wildfires
4,11,NaN,NaN,Typhoon Soudelor kills 28 in China and Taiwan
...,...,...,...,...
3258,10861,NaN,NaN,EARTHQUAKE SAFETY LOS ANGELES ÛÒ SAFETY FASTE...
3259,10865,NaN,NaN,Storm in RI worse than last hurricane. My city...
3260,10868,NaN,NaN,Green Line derailment in Chicago http://t.co/U...
3261,10874,NaN,NaN,MEG issues Hazardous Weather Outlook (HWO) htt...


In [8]:
test.text.nunique()

3243

In [9]:
train = train.drop_duplicates(subset = ['text'])
print(train.shape)

X = train.drop('target',axis=1)
y = train.target
X_train, X_test, y_train, y_test = train_test_split( X, y, test_size=0.2, random_state=42)

(7503, 5)


In [10]:
train['target'].value_counts()

0    4305
1    3198
Name: target, dtype: int64

In [11]:
train[train['target']==1].text.tolist()

['Our Deeds are the Reason of this #earthquake May ALLAH Forgive us all',
 'Forest fire near La Ronge Sask. Canada',
 "All residents asked to 'shelter in place' are being notified by officers. No other evacuation or shelter in place orders are expected",
 '13,000 people receive #wildfires evacuation orders in California ',
 'Just got sent this photo from Ruby #Alaska as smoke from #wildfires pours into a school ',
 '#RockyFire Update => California Hwy. 20 closed in both directions due to Lake County fire - #CAfire #wildfires',
 '#flood #disaster Heavy rain causes flash flooding of streets in Manitou, Colorado Springs areas',
 "I'm on top of the hill and I can see a fire in the woods...",
 "There's an emergency evacuation happening now in the building across the street",
 "I'm afraid that the tornado is coming to our area...",
 'Three people died from the heat wave so far',
 'Haha South Tampa is getting flooded hah- WAIT A SECOND I LIVE IN SOUTH TAMPA WHAT AM I GONNA DO WHAT AM I GONNA 

In [12]:
train[train['target']==0].text.tolist()

["What's up man?",
 'I love fruits',
 'Summer is lovely',
 'My car is so fast',
 'What a goooooooaaaaaal!!!!!!',
 'this is ridiculous....',
 'London is cool ;)',
 'Love skiing',
 'What a wonderful day!',
 'LOOOOOOL',
 "No way...I can't eat that shit",
 'Was in NYC last week!',
 'Love my girlfriend',
 'Cooool :)',
 'Do you like pasta?',
 'The end!',
 'We always try to bring the heavy. #metal #RT http://t.co/YAo1e0xngw',
 'Crying out for more! Set me ablaze',
 'On plus side LOOK AT THE SKY LAST NIGHT IT WAS ABLAZE http://t.co/qqsmshaJ3N',
 "@PhDSquares #mufc they've built so much hype around new acquisitions but I doubt they will set the EPL ablaze this season.",
 'Ablaze for you Lord :D',
 'Check these out: http://t.co/rOI2NSmEJJ http://t.co/3Tj8ZjiN21 http://t.co/YDUiXEfIpE http://t.co/LxTjc87KLS #nsfw',
 "on the outside you're ablaze and alive\nbut you're dead inside",
 'Had an awesome time visiting the CFC head office the ancop site and ablaze. Thanks to Tita Vida for taking care of 

All the tweets are in english?  RTA: ....


In [13]:
for t in X_train.text.unique():
  errors = []
  try:
    text = Text(t)
    if text.language.code!='en':
      print("Language Detected: Code={}, Name={}\n".format(text.language.code, text.language.name))
      print(t)
  except:
    errors.append(t)
errors

Detector is not able to detect the language reliably.
Detector is not able to detect the language reliably.
Detector is not able to detect the language reliably.
Detector is not able to detect the language reliably.
Detector is not able to detect the language reliably.
Detector is not able to detect the language reliably.
Detector is not able to detect the language reliably.
Detector is not able to detect the language reliably.
Detector is not able to detect the language reliably.
Detector is not able to detect the language reliably.
Detector is not able to detect the language reliably.
Detector is not able to detect the language reliably.
Detector is not able to detect the language reliably.
Detector is not able to detect the language reliably.
Detector is not able to detect the language reliably.
Detector is not able to detect the language reliably.
Detector is not able to detect the language reliably.
Detector is not able to detect the language reliably.
Detector is not able to dete

Language Detected: Code=sco, Name=Scots

@Deeeznvtzzz bring the body bags tho
Language Detected: Code=da, Name=Danish

#Sismo DETECTADO #JapÌ_n 15:41:07 Seismic intensity 0 Iwate Miyagi JST #?? http://t.co/gMoUl9zQ2Q


Detector is not able to detect the language reliably.
Detector is not able to detect the language reliably.
Detector is not able to detect the language reliably.
Detector is not able to detect the language reliably.
Detector is not able to detect the language reliably.
Detector is not able to detect the language reliably.
Detector is not able to detect the language reliably.
Detector is not able to detect the language reliably.
Detector is not able to detect the language reliably.
Detector is not able to detect the language reliably.
Detector is not able to detect the language reliably.
Detector is not able to detect the language reliably.
Detector is not able to detect the language reliably.
Detector is not able to detect the language reliably.
Detector is not able to detect the language reliably.
Detector is not able to detect the language reliably.
Detector is not able to detect the language reliably.
Detector is not able to detect the language reliably.
Detector is not able to dete

Language Detected: Code=war, Name=Waray

@5SOSFamUpdater social casualty
Language Detected: Code=is, Name=Icelandic

@LoneWolffur BRUH *dies*
Language Detected: Code=la, Name=Latin

@Blizzard_Gamin ight
Language Detected: Code=sco, Name=Scots

she's a suicide bomb
Language Detected: Code=war, Name=Waray

Social Casualty #MTVHottest 5SOS
Language Detected: Code=nn, Name=Norwegian Nynorsk

Fatality!
Language Detected: Code=pt, Name=Portuguese

#RolandoNaBeats: Ellie Goulding - My Blood | Acesse nosso site para ouvir! http://t.co/Zk69uGXMT8
Language Detected: Code=st, Name=Southern Sotho

SANDSTORM!!! WOO HOO!!
Language Detected: Code=sk, Name=Slovak

Bloody insomnia again! Grrrr!! #Insomnia


Detector is not able to detect the language reliably.
Detector is not able to detect the language reliably.
Detector is not able to detect the language reliably.
Detector is not able to detect the language reliably.
Detector is not able to detect the language reliably.
Detector is not able to detect the language reliably.
Detector is not able to detect the language reliably.
Detector is not able to detect the language reliably.
Detector is not able to detect the language reliably.
Detector is not able to detect the language reliably.
Detector is not able to detect the language reliably.
Detector is not able to detect the language reliably.
Detector is not able to detect the language reliably.
Detector is not able to detect the language reliably.
Detector is not able to detect the language reliably.
Detector is not able to detect the language reliably.
Detector is not able to detect the language reliably.
Detector is not able to detect the language reliably.
Detector is not able to dete

...

Detector is not able to detect the language reliably.
Detector is not able to detect the language reliably.
Detector is not able to detect the language reliably.
Detector is not able to detect the language reliably.
Detector is not able to detect the language reliably.
Detector is not able to detect the language reliably.
Detector is not able to detect the language reliably.
Detector is not able to detect the language reliably.
Detector is not able to detect the language reliably.
Detector is not able to detect the language reliably.
Detector is not able to detect the language reliably.
Detector is not able to detect the language reliably.
Detector is not able to detect the language reliably.
Detector is not able to detect the language reliably.
Detector is not able to detect the language reliably.
Detector is not able to detect the language reliably.
Detector is not able to detect the language reliably.
Detector is not able to detect the language reliably.
Detector is not able to dete

Language Detected: Code=sco, Name=Scots

@Truly_Stings Yo Dm me
Language Detected: Code=la, Name=Latin

@BenignoVito @LibertyBell1000 HILLARYMASS MURDERER.
Language Detected: Code=sk, Name=Slovak

@iK4LEN Sirens was cancelled.
Language Detected: Code=nn, Name=Norwegian Nynorsk

Fatality ????
Language Detected: Code=it, Name=Italian

@pxnatosil @RenuncieDilma  Fatality!
Language Detected: Code=da, Name=Danish

Hellfire Gargoyle Hoof coil http://t.co/2ii3Brc7NX
Language Detected: Code=sco, Name=Scots

'Cooler than Freddie Jackson sippin' a milkshake in a snowstorm'
Language Detected: Code=jw, Name=Javanese

#UdhampurAgain 2 terrorist shot dead.. #Udhampur
Language Detected: Code=da, Name=Danish

@TSUNAMI_nopeach ?????? I'm weak af


Detector is not able to detect the language reliably.
Detector is not able to detect the language reliably.
Detector is not able to detect the language reliably.
Detector is not able to detect the language reliably.
Detector is not able to detect the language reliably.
Detector is not able to detect the language reliably.
Detector is not able to detect the language reliably.
Detector is not able to detect the language reliably.
Detector is not able to detect the language reliably.
Detector is not able to detect the language reliably.
Detector is not able to detect the language reliably.
Detector is not able to detect the language reliably.
Detector is not able to detect the language reliably.
Detector is not able to detect the language reliably.
Detector is not able to detect the language reliably.
Detector is not able to detect the language reliably.
Detector is not able to detect the language reliably.
Detector is not able to detect the language reliably.
Detector is not able to dete

Language Detected: Code=om, Name=Oromo

Seek help warra #MetroFmTalk
Language Detected: Code=tn, Name=Tswana

Let her go - Passenger
Language Detected: Code=sco, Name=Scots

that's it val is dead im suing
Language Detected: Code=nn, Name=Norwegian Nynorsk

@Babybackreeve FATALITY!!!!!!!!!!
Language Detected: Code=un, Name=un

lmfao fucking luis hhahaha im dead


[]

# 1. SMALL WAY: Normalization (format rules)
## 1.1. REGEX https://www.w3schools.com/python/python_regex.asp <br>
Ex. dates SEBASTIAN<br>
Countries: leidy<br>
nombres propios: FRANCISCO<br>
palabras compuestas (-): SEBASTIAN human-center<br>
apostrofes -> juntar Ex. it's: FAVIO<br>
hipervinculos: LEIDY<br>
HASHTAGS: THOMAS #: punctuation?<br>
EMOJIS- caracteres especiales: THOMAS: @users<br>


RESULT: 
INPUT: texto, texto, texto
CORPUS_1: Array of clean documents  (list before)['texto','texto']

In [14]:
tweets_train = X_train.text.unique()
tweets_test = X_test.text.unique()

In [16]:
X_test.head()

,id,keyword,location,text
2330,3351,demolished,"Paterson, New Jersey",Three Homes Demolished in Unrecognized Arab Vi...
3391,4854,evacuation,NaN,Reid Lake fire prompts campground evacuation o...
3363,4814,evacuation,NaN,FAAN orders evacuation of abandoned aircraft a...
4632,6583,injury,Baltimore,Ngata on injury list at start of practice for ...
7588,10842,NaN,NaN,BREAKING: #ISIS claims responsibility for mosq...


#### Apostrophes 

In [15]:
#Favio's apostrophes
#FIRST APROACH
#Erasing the apostrophes from the tweets and putting instead both words
def decontracted1(phrase) -> str:
    # specific
    phrase = re.sub(r"won\'t", "will not", phrase)
    phrase = re.sub(r"can\'t", "can not", phrase)

    # spaces
    phrase = re.sub(r"' ", " ", phrase)
    phrase = re.sub(r" '", " ", phrase)

    # general
    phrase = re.sub(r"n\'t", " not", phrase)
    phrase = re.sub(r"\'re", " are", phrase)
    phrase = re.sub(r"\'s", " is", phrase)
    phrase = re.sub(r"\'d", " would", phrase)
    phrase = re.sub(r"\'ll", " will", phrase)
    phrase = re.sub(r"\'t", " not", phrase)
    phrase = re.sub(r"\'ve", " have", phrase)
    phrase = re.sub(r"\'m", " am", phrase)

    return phrase

#SECOND APROACH
#Erasing the apostrophes from the tweets
def decontracted2(phrase) -> str:
    phrase = re.sub(r"'", "", phrase)
    return phrase

#Execution with the dataset for FIRST APROACH

def dataset_decontracted(lista):

    nueva_lista = []
    for tweet in lista:
        new_tweet = decontracted1(tweet)
        nueva_lista.append(new_tweet)

    print(len(nueva_lista),len(lista))
    return nueva_lista


In [16]:
corpus_train = dataset_decontracted(tweets_train)
corpus_test = dataset_decontracted(tweets_test)

6002 6002
1501 1501


#### Hipervinculos

In [17]:
#HIPERVINCULOS
def hip(corpus):
  corpus=[re.sub(r'http\S+', '',i) for i in corpus]
  print(len(corpus))
  return corpus

In [18]:
corpus_train = hip(corpus_train)
corpus_test = hip(corpus_test)

6002
1501


#### Dates

In [19]:
#DATES SEBASTIAN

def dates(texts):
  lista = []
  listan =[]
  for text in texts:

    try:
      var = False
      par = parser.parse(text, fuzzy_with_tokens=True)
      #print(par)
      #print(len(par))

      for i in range(len(par)):
        if i == 0:
          aux = par[i].strftime(" %Y|%m|%d ")
          #print(aux)

        if i == 1:
          lista = list(par[i])
          #print(lista)
          #print(par[i])
          #print(len(lista))

          if len(lista) == 0:
            lista.append("")

          if ("" or " " or "  ") not in lista:
            lista.append(" ")

          for j in range(len(lista)):
              
            #print(len(lista))
            if (lista[j] == " " or lista[j] == "" or lista[j] == "  ") and var == False:
              lista[j] = aux
              var = True
        lista = "".join(lista)
      listan.append(lista)
      #print(aux)    

    except:
      listan.append(text)
  print(len(listan))    
  return listan

In [20]:
corpus_train = dates(corpus_train)
corpus_test = dates(corpus_test)

/usr/local/lib/python3.7/dist-packages/dateutil/parser/_parser.py:1212: UnknownTimezoneWarning: tzname ET identified but not understood.  Pass `tzinfos` argument in order to correctly return a timezone-aware datetime.  In a future version, this will raise an exception.
  category=UnknownTimezoneWarning)
/usr/local/lib/python3.7/dist-packages/dateutil/parser/_parser.py:1212: UnknownTimezoneWarning: tzname CST identified but not understood.  Pass `tzinfos` argument in order to correctly return a timezone-aware datetime.  In a future version, this will raise an exception.
  category=UnknownTimezoneWarning)
/usr/local/lib/python3.7/dist-packages/dateutil/parser/_parser.py:1212: UnknownTimezoneWarning: tzname EB identified but not understood.  Pass `tzinfos` argument in order to correctly return a timezone-aware datetime.  In a future version, this will raise an exception.
  category=UnknownTimezoneWarning)
/usr/local/lib/python3.7/dist-packages/dateutil/parser/_parser.py:1212: UnknownTimez

6002
1501


/usr/local/lib/python3.7/dist-packages/dateutil/parser/_parser.py:1212: UnknownTimezoneWarning: tzname SIXTH identified but not understood.  Pass `tzinfos` argument in order to correctly return a timezone-aware datetime.  In a future version, this will raise an exception.
  category=UnknownTimezoneWarning)
/usr/local/lib/python3.7/dist-packages/dateutil/parser/_parser.py:1212: UnknownTimezoneWarning: tzname BKB identified but not understood.  Pass `tzinfos` argument in order to correctly return a timezone-aware datetime.  In a future version, this will raise an exception.
  category=UnknownTimezoneWarning)


#### Caracteres

In [21]:
#Thomas

#Removes @, # and irritating bold formatting
def characters (corpus):
  #thomas_list = [' '.join(re.sub("(@[A-Za-z0-9]+)|([^0-9A-Za-z \t])|(\w+:\/\/\S+)"," ",x.encode("ascii","ignore").decode()).split()) for x in corpus]
  thomas_list = [' '.join(re.sub("(@[A-Za-z0-9]+)|([^0-9A-Za-z \t])|(\w+:\/\/\S+)|(\d+)|(\|)"," ",x.encode("ascii","ignore").decode()).split()) for x in corpus]
  print(len(thomas_list))
  return thomas_list

In [22]:
corpus_train = characters(corpus_train)
corpus_test = characters(corpus_test)

6002
1501


#### Lower Case

In [23]:
def lowercase (corpus):
  corpus = [x.lower() for x in corpus]
  print(len(corpus))
  return corpus

In [24]:
corpus_train = lowercase(corpus_train)
corpus_test = lowercase(corpus_test)

6002
1501


#### Proper Names

In [25]:
#Juan
#Delete title prefix
twitterNameCleaner = ["Mr", "Mrs", "Miss", "Ms", "Mx", "Sir", "Dr", "Cllr", "Lady", "Lord"]
nlp = en_core_web_md.load()

def proper_names(corpus):
  
  for prefix in twitterNameCleaner:
    corpus = [' '.join(re.sub(prefix," ",x.encode("ascii","ignore").decode()).split()) for x in corpus]
  
  for t in corpus:
    doc = nlp(t)
    for ent in doc.ents:
      if ent.label_=='PERSON':
        corpus=[re.sub(ent.text, '',i) for i in corpus]
  print(len(corpus))
  return corpus

In [26]:
#doc = nlp(' '.join(juan_list_1))
#labels = [x.label_ for x in doc.ents]
#Counter(labels)

In [27]:
#there are 1054 organisation names in all the tweets


In [28]:
#a = [(X.text, X.label_) for X in doc.ents]
#organisation_names = []
#people_name = []
#location_name = []

#for i in range(len(a)):
#  if a[i][1] == 'ORG':
#    organisation_names.append(a[i][0])

#  elif a[i][1] == 'PERSON':
#    people_name.append(a[i][0])

#  elif a[i][1] == 'LOC':
#    location_name.append(a[i][0])

In [29]:
corpus_train = proper_names(corpus_train)
corpus_test = proper_names(corpus_test)

6002
1501


#### Countries

In [ ]:
nlp = en_core_web_sm.load()
l2 = []
for t in corpus:
  doc = nlp(t)
  for ent in doc.ents:
    if ent.label_=='GPE' and ent.text not in l2:
      l2.append(ent.text)
l2

['cfornia',
 'lake county',
 'nitou',
 'colorado',
 'south tampa',
 'florida',
 'myanr',
 'london',
 'nyc',
 'aba',
 'abia',
 'santa cruz',
 'diyala',
 'los angeles',
 'ashville',
 'kenya',
 'thinng',
 'france',
 'brazil',
 'colombia',
 'britain',
 'iran',
 'camden',
 'israel',
 'arizona',
 'cairo',
 'xbox',
 'dvd',
 'russia',
 'america',
 'japan',
 'cago',
 'arson',
 'tennessee',
 'long beach',
 'salem',
 'nashville',
 'italy',
 'kara',
 'hea',
 'noh',
 'us',
 'india',
 'santa ria',
 'ps vita',
 'gaza',
 'greece',
 'bitcoin',
 'fukushi',
 'new hampshire',
 'atlanta',
 'orchardalley',
 'les',
 'festac town',
 'memphis',
 'ohio',
 'detroit',
 'heahstone',
 'imouto',
 'porridge',
 'acesse',
 'tokyo',
 'twos',
 'nitroglycerin',
 'canada',
 'meek mill',
 'charlotte',
 'handbags',
 'fairfax',
 'namjoon',
 'zumiez',
 'libya',
 'noh rica',
 'u s',
 'hiroshi',
 'syria',
 'tang',
 'turkey',
 'kanye',
 'qendil',
 'poland',
 'sioux city',
 'england',
 'outta',
 'awesomeeeeeeee',
 'troy',
 'bright

In [ ]:
len(l2)

376

#### Multiple Spaces

In [30]:
def spaces(corpus):
  corpus = [re.sub("\s\s+" , " ", x) for x in corpus]
  print(len(corpus))
  return corpus

In [31]:
corpus_train = spaces(corpus_train)
corpus_test = spaces(corpus_test)

6002
1501


## 1.3. TFIDF SKLEARN CON 1.1 Y 1.2

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

vectorizer = TfidfVectorizer(max_df=0.99, stop_words="english")
X = vectorizer.fit_transform(corpus)
cols = vectorizer.get_feature_names()

#print(vectorizer.get_feature_names_out())
print(X.shape)

(7503, 11830)


/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


In [ ]:
X1 = X.toarray()
df_tfidf = pd.DataFrame(X1, columns = cols, index = corpus)

In [ ]:
df_tfidf.head(10)

,aa,aaaa,aaaaaaallll,aaaaaand,aaaan,aaarrrgghhh,aan,aannnnd,aar,ab,aba,abandon,abandoning,abbandon,abbruchsimulator,abbs,abc,abceyewitness,abcnews,abia,ability,abject,ablaze,able,aboard,aboion,aboions,abomb,abomination,abouts,abq,abs,absence,absolute,absolutely,abstract,absurd,absurdly,abus,abuse,...,yzf,zabai,zach,zan,zar,zda,ze,zeal,zehrs,zenit,zergele,zero,zeros,zhejiang,zicac,zimbabwe,zing,zionism,zionist,zionists,zip,zipp,zipper,zippers,zippnews,zippoline,ziristan,zmne,zombie,zombies,zone,zones,zoom,zotar,zrnf,zss,zumiez,zurich,zy,zzzz
our des are the reason of this eahquake allah forgive us all,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
forest fire near la ronge sask canada,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
all residents ask to shelter in place are being notifi by officers no other evacuation or shelter in place orders are pect,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
people receive wildfires evacuation orders in cfornia,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
just got sent this photo from as smoke from wildfires pours into a school,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
rockyfire update cfornia hwy clos in both directions due to lake county fire cire wildfires,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
flood disaster heavy rain causes flash flooding of streets in nitou colorado springs areas,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
i am on top of the hill and i can see a fire in the woods,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
there is an emergency evacuation happening now in the building ac the street,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0

In [ ]:
vectorizer.vocabulary_

{'des': 2726,
 'reason': 8359,
 'eahquake': 3205,
 'allah': 262,
 'forgive': 3944,
 'forest': 3936,
 'near': 6749,
 'la': 5722,
 'ronge': 8797,
 'sask': 8976,
 'canada': 1493,
 'residents': 8565,
 'ask': 546,
 'shelter': 9275,
 'place': 7706,
 'notifi': 6940,
 'officers': 7097,
 'evacuation': 3520,
 'orders': 7230,
 'pect': 7516,
 'people': 7541,
 'receive': 8375,
 'wildfires': 11494,
 'cfornia': 1699,
 'just': 5588,
 'got': 4358,
 'sent': 9165,
 'photo': 7613,
 'smoke': 9542,
 'pours': 7896,
 'school': 9037,
 'rockyfire': 8773,
 'update': 11105,
 'hwy': 4982,
 'clos': 1913,
 'directions': 2869,
 'lake': 5740,
 'county': 2296,
 'cire': 1835,
 'flood': 3857,
 'disaster': 2885,
 'heavy': 4682,
 'rain': 8264,
 'causes': 1625,
 'flash': 3835,
 'flooding': 3858,
 'streets': 9992,
 'nitou': 6863,
 'colorado': 2013,
 'springs': 9786,
 'areas': 484,
 'hill': 4765,
 'woods': 11600,
 'emergency': 3317,
 'happening': 4580,
 'building': 1352,
 'ac': 42,
 'street': 9989,
 'raid': 8256,
 'tornado': 

In [ ]:
(df_tfidf != 0).sum(1).sort_values()

                                                                                                                                        0
 is fire                                                                                                                                0
                                                                                                                                        0
 please o                                                                                                                               0
what is up n                                                                                                                            0
                                                                                                                                       ..
rite now n a tlk widout nuh curfew long side aka cum fi steal di show itz a rubbery di whole a dem fi knw sound it music               17
ux fail of emv people nt to inse a

# 2. LARGE WAY

## 2.1 FUNCTION

In [32]:
def tf_idf(corpus, dataset, labels):

  #LEMA
  diccionarioL = set()
  #diccionarioT = set()
  diccionarioSW = set()
  diccionarioTokens = {}
  # Ejemplo tokenizador por palabra, stop words(no se eliminan, solo las identifica), NO hace normalizacion y (Hace lematizacion no stemming!)
  #nlp = spacy.load("en_core_web_md")
  for txt in corpus:
    li=[]
    doc = nlp(txt)
    #print(doc.vector)
    
    for token in doc:
      if (token.is_alpha or token.is_digit) and not token.is_stop and not token.is_punct:
          diccionarioL.add(token.lemma_)
          li.append(token.lemma_)
          #diccionarioT.add(token.text)
      if token.is_stop:
        diccionarioSW.add(token.text)

    diccionarioTokens[txt] = dict(Counter(li))

  # FRECUENCIAS

  count = pd.DataFrame(columns=diccionarioL, index = corpus)

  for i in diccionarioTokens:
    
    for l in diccionarioTokens[i]:
        count.loc[i,l] = diccionarioTokens[i][l]
  count=count.fillna(0)
  print(count.shape)

  tokens = count.columns 

  #count['id'] = dataset['id'].tolist()
  #TODO// Hacer despues de
  #count = (count.loc[count[tokens].sum(1) > 1 ])

  tf = count[tokens].div(count[tokens].sum(1), axis = 0 )
  print(tf.shape)
  #IDF
  N = tf.shape[0]
  #En cuantos tweets aparece cada token
  df = tf.astype(bool).sum(axis = 0)
  idf = pd.DataFrame(columns = ['value'], index = tf.columns)

  i = 0
  # Numero de tokens
  for i in range(len(idf['value'])):
      idf['value'][i] = np.log(N/(df[i] + 1))
  print(idf.shape)
  tf_idf = tf*idf.T.iloc[0]

  tf_idf['Real_Target'] = labels.tolist()
  #tf_idf['id'] = count['id']
  print(tf_idf.shape)
  #tf_idf_label = dataset[["id","target"]].rename(columns={'target':'Real_Target'},axis=1)

  #Final_tf_idf = pd.merge(tf_idf, tf_idf_label, on="id", how="left")
  #print(Final_tf_idf)

  #return tokens, Final_tf_idf stopWords
  return tokens, tf_idf, diccionarioSW


In [33]:
#TODO (primera solucion):// Separar diccionario de tf idf. con el train original

In [34]:
tokens_Train , Final_tf_idf_Train, StopWordsTrain = tf_idf(corpus_train, X_train, y_train)
tokens_Test , Final_tf_idf_Test, StopWordsTest = tf_idf(corpus_test, X_test, y_test)

(6002, 9231)
(6002, 9231)
(9231, 1)
(6002, 9232)
(1501, 4136)
(1501, 4136)
(4136, 1)
(1501, 4137)


In [37]:
len(corpus_train), X_train.shape

(7503, (6002, 4))

In [35]:
Final_tf_idf_Train  = (Final_tf_idf_Train.loc[Final_tf_idf_Train[Final_tf_idf_Train.columns[:-1]].gt(0).sum(axis=1) > 1 ])
Final_tf_idf_Train

,destroyer,measle,eland,unarm,prob,testimony,line,tem,saudie,blacken,literally,usg,string,homicide,etoffe,sweaty,rubybot,douchebag,guest,snapchatselfie,womb,breangpolice,davis,trynna,newbeginningsanilrescue,bulletin,bmw,asts,photo,water,holy,cannon,month,organic,collection,word,overhead,baystate,condition,hrsto,...,albeit,agency,fts,infectiousdiseases,impulse,nsehra,contruction,dustpig,sea,pepperoni,stan,well,calamity,oregon,campbell,feral,fact,function,southdown,comprehensive,impressive,brooo,sinng,blaze,funtenna,moth,trekker,threesome,improv,musician,sandbox,girls,droid,upgrade,virus,tourist,practitioner,lone,series,Real_Target
for the people who di in hun periments by unit of japanese military,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1
potential storm surge flooding p by national hurricane center,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1
sometimes your cheeness b through my computer screen and i recoil in fear,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
can still see it blank pression cheeks clapping in my face room playing subtlety in the back yet drowning out all the sound,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
clipuri the hobbit the desolation of sug poze thehobbit,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
refu oil spill have been costlier bigger than project,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1
lennaires disease what is being done to stop deadly outbreak the worst ever outbreak of le losangeles,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1
bangalore rdit updates content policy promises to quarantine tremely offensive communities startups in,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
biry young blood,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [134]:
Final_tf_idf_Train.sum(axis=1)

for the people who di in hun periments by unit of japanese military                                                            6.215079
potential storm surge flooding p by national hurricane center                                                                  6.276576
sometimes your cheeness b through my computer screen and i recoil in fear                                                      5.661765
can still see it blank pression cheeks clapping in my face room playing subtlety in the back yet drowning out all the sound    6.020653
 clipuri the hobbit the desolation of sug poze thehobbit                                                                       6.227121
                                                                                                                                 ...   
refu oil spill have been costlier bigger than project                                                                          6.106932
lennaires disease what is being done to stop dea

In [70]:
s=Final_tf_idf_Train[Final_tf_idf_Train.columns[:-1]].gt(0).sum(axis=1)
s[s == 0]

did you see with     0
                     0
 back up             0
did you see with     0
                     0
dtype: int64

In [72]:
Final_tf_idf_Train.Real_Target.value_counts()

0    3331
1    2555
Name: Real_Target, dtype: int64

## 2.2. Tokenización y Lemmatización (spacy) 
INPUT: CORPUS_1 ['tweet1', 'tweet2','text3', 'text4']<br> 

TODO: stopwords, lematizar.....<br>

OUTPUT: DICT, ARRAY : TWEET X TOKENS: Ex.  {'Flood: Two people dead 60 houses destroyed in Kaduna: Two people have been reportedly killed and 60 houses ut... http://t.co/JixScpMdUD': [tokens]}<br>
Ex2:
1 : [tokens],
2 : [X, Y, Z],
3 : [A,A,A,A,A,A]




In [ ]:
#Lematizacion
#Retorna un diccionario de diccionarios: llaves= documento: valores = diccionario de (llaves) terminos con su conteo(valores)

diccionarioL = set()
diccionarioT = set()
diccionarioTokens = {}
# Ejemplo tokenizador por palabra, stop words(no se eliminan, solo las identifica), NO hace normalizacion y (Hace lematizacion no stemming!)
#nlp = spacy.load("en_core_web_md")
for txt in corpus:
  li=[]
  doc = nlp(txt.lower())
  #print(doc.vector)
  
  for token in doc:
    if (token.is_alpha or token.is_digit) and not token.is_stop and not token.is_punct:
        diccionarioL.add(token.lemma_)
        li.append(token.lemma_)
        diccionarioT.add(token.text)

  diccionarioTokens[txt]=dict(Counter(li))

In [ ]:
# Build dataframe of docs x terms -> only count
count = pd.DataFrame(columns=diccionarioL, index = corpus)

for i in diccionarioTokens:
  
  for l in diccionarioTokens[i]:
      count.loc[i,l] = diccionarioTokens[i][l]
count=count.fillna(0)
count

,gfe,american,deluxe,telerketing,elk,newbeginningsanilrescue,aicbomb,reflection,ph,metal,outrage,faroe,burger,delay,gosh,hardy,prison,wifi,discuss,gic,villa,gas,temecula,new,wolf,horizon,quarrel,plosive,promo,scar,os,cutest,coiff,crew,yzf,bordento,gal,ndle,captureyyc,beg,...,ferrell,ransomre,subtlety,highland,rds,bluio,notion,hamptonroadsfor,pronounce,engag,omron,service,faq,joseph,throwback,board,dealbreaker,pande,senator,plot,myth,bow,lassic,humboldt,accidentally,ishe,sail,suruc,reveng,trial,dk,ls,register,fuckboy,debris,unbelievably,tos,nowplay,slowpoke,dyersburg
our des are the reason of this eahquake allah forgive us all,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
forest fire near la ronge sask canada,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
all residents ask to shelter in place are being notifi by officers no other evacuation or shelter in place orders are pect,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
people receive wildfires evacuation orders in cfornia,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
just got sent this photo from as smoke from wildfires pours into a school,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
worldnews fallen powerlines on g link tram update fire crews have evacuat up to passengers who were tr,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
on the flip side i am at and there is a bomb and everyone had to evacuate so stay tun if i blow up or not,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
suicide bomber lls in saudi security site mosque reuters via world google news ll,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
two giant cranes holding a bridge collapse into nearby homes,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [ ]:
count.shape

(7503, 10465)

In [ ]:
(count != 0).sum(1).sort_values()

did you see with                                                                                                                  0
                                                                                                                                  0
 back up                                                                                                                          0
                                                                                                                                  0
it what                                                                                                                           0
                                                                                                                                 ..
england east coast dogger bank westrd seismic survey in progress by m v western regent towing a metre long cable within area     18
rite now n a tlk widout nuh curfew long side aka cum fi steal di show itz a 

In [ ]:
count.sum(axis=0)

gfe              1
american        25
deluxe           1
telerketing      1
elk              1
                ..
unbelievably     1
tos              2
nowplay          1
slowpoke         1
dyersburg        1
Length: 10465, dtype: int64

In [ ]:
l = count.columns

In [ ]:
l

Index(['gfe', 'american', 'deluxe', 'telerketing', 'elk',
       'newbeginningsanilrescue', 'aicbomb', 'reflection', 'ph', 'metal',
       ...
       'dk', 'ls', 'register', 'fuckboy', 'debris', 'unbelievably', 'tos',
       'nowplay', 'slowpoke', 'dyersburg'],
      dtype='object', length=10465)

In [ ]:
#count['target'] = train.target
count['id'] = train['id'].tolist()
count

,gfe,american,deluxe,telerketing,elk,newbeginningsanilrescue,aicbomb,reflection,ph,metal,outrage,faroe,burger,delay,gosh,hardy,prison,wifi,discuss,gic,villa,gas,temecula,new,wolf,horizon,quarrel,plosive,promo,scar,os,cutest,coiff,crew,yzf,bordento,gal,ndle,captureyyc,beg,...,ransomre,subtlety,highland,rds,bluio,notion,hamptonroadsfor,pronounce,engag,omron,service,faq,joseph,throwback,board,dealbreaker,pande,senator,plot,myth,bow,lassic,humboldt,accidentally,ishe,sail,suruc,reveng,trial,dk,ls,register,fuckboy,debris,unbelievably,tos,nowplay,slowpoke,dyersburg,id
our des are the reason of this eahquake allah forgive us all,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1
forest fire near la ronge sask canada,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,4
all residents ask to shelter in place are being notifi by officers no other evacuation or shelter in place orders are pect,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,5
people receive wildfires evacuation orders in cfornia,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,6
just got sent this photo from as smoke from wildfires pours into a school,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,7
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
worldnews fallen powerlines on g link tram update fire crews have evacuat up to passengers who were tr,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,10863
on the flip side i am at and there is a bomb and everyone had to evacuate so stay tun if i blow up or not,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,10864
suicide bomber lls in saudi security site mosque reuters via world google news ll,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,10866
two giant cranes holding a bridge collapse into nearby homes,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,10869


In [ ]:
count = (count.loc[count[l].sum(1) > 1 ])
count.shape

(7362, 10466)

In [ ]:
tf = count[l].div(count[l].sum(1), axis = 0 )
tf

,gfe,american,deluxe,telerketing,elk,newbeginningsanilrescue,aicbomb,reflection,ph,metal,outrage,faroe,burger,delay,gosh,hardy,prison,wifi,discuss,gic,villa,gas,temecula,new,wolf,horizon,quarrel,plosive,promo,scar,os,cutest,coiff,crew,yzf,bordento,gal,ndle,captureyyc,beg,...,ferrell,ransomre,subtlety,highland,rds,bluio,notion,hamptonroadsfor,pronounce,engag,omron,service,faq,joseph,throwback,board,dealbreaker,pande,senator,plot,myth,bow,lassic,humboldt,accidentally,ishe,sail,suruc,reveng,trial,dk,ls,register,fuckboy,debris,unbelievably,tos,nowplay,slowpoke,dyersburg
our des are the reason of this eahquake allah forgive us all,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
forest fire near la ronge sask canada,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
all residents ask to shelter in place are being notifi by officers no other evacuation or shelter in place orders are pect,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
people receive wildfires evacuation orders in cfornia,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
just got sent this photo from as smoke from wildfires pours into a school,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
worldnews fallen powerlines on g link tram update fire crews have evacuat up to passengers who were tr,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.083333,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
on the flip side i am at and there is a bomb and everyone had to evacuate so stay tun if i blow up or not,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
suicide bomber lls in saudi security site mosque reuters via world google news ll,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.

In [ ]:
d = {'col1': [1, 1], 'col2': [1, 0], 'col3': [0, 3], 'col4': [0, 5]}
df = pd.DataFrame(data=d, index=[0,1])
df1 = df.sum(1)
#print(df.head())
df.astype(bool).sum(axis=0)#df.div(df.sum(1), axis = 0 )
for i in df.index:
    print(i)

0
1


In [ ]:
(tf != 0).sum(0).sort_values()

kno          0
err          0
favorit      0
sunburn      0
ember        0
          ... 
amp        289
ter        311
fire       319
like       331
s          690
Length: 10465, dtype: int64

In [ ]:
#IDF Calculate
#Numero de tweets
N = tf.shape[0]
#En cuantos tweets aparece cada token
df = tf.astype(bool).sum(axis = 0)
idf = pd.DataFrame(columns = ['value'], index = tf.columns)

i = 0
# Numero de tokens
for i in range(len(idf['value'])):
    idf['value'][i] = np.log(N/(df[i] + 1))
print(len(idf))

10465


In [ ]:
tf_idf = tf*idf.T.iloc[0]

example of tf idf calculus

In [ ]:
tf.iloc[1].fire,idf.T.iloc[0].fire,tf.iloc[1].fire*idf.T.iloc[0].fire

(0.14285714285714285, 3.1357659181451942, 0.44796655973502775)

In [ ]:
tf.iloc[1].sort_values()

gfe            0.000000
instruction    0.000000
finse          0.000000
pineview       0.000000
mlb            0.000000
                 ...   
forest         0.142857
ronge          0.142857
fire           0.142857
near           0.142857
sask           0.142857
Name: forest fire near la ronge sask canada, Length: 10465, dtype: float64

In [ ]:
tf_idf.iloc[1].fire

0.44796655973502775

In [ ]:
tf_idf.shape

(7362, 10465)

In [ ]:
tf_idf['id'] = count['id']

In [ ]:
tf_idf_label = train[["id","target"]]
tf_idf_label.head

<bound method NDFrame.head of          id  target
0         1       1
1         4       1
2         5       1
3         6       1
4         7       1
...     ...     ...
7604  10863       1
7605  10864       1
7606  10866       1
7608  10869       1
7612  10873       1

[7503 rows x 2 columns]>

In [ ]:
tf_idf_label.rename(columns={'target':'Real_Target'}, inplace = True)
tf_idf_label.head()

/usr/local/lib/python3.7/dist-packages/pandas/core/frame.py:4308: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


,id,Real_Target
0,1,1
1,4,1
2,5,1
3,6,1
4,7,1


In [ ]:
Final_tf_idf = pd.merge(tf_idf, tf_idf_label, on="id", how="left")
Final_tf_idf.head(5)

,gfe,american,deluxe,telerketing,elk,newbeginningsanilrescue,aicbomb,reflection,ph,metal,outrage,faroe,burger,delay,gosh,hardy,prison,wifi,discuss,gic,villa,gas,temecula,new,wolf,horizon,quarrel,plosive,promo,scar,os,cutest,coiff,crew,yzf,bordento,gal,ndle,captureyyc,beg,...,subtlety,highland,rds,bluio,notion,hamptonroadsfor,pronounce,engag,omron,service,faq,joseph,throwback,board,dealbreaker,pande,senator,plot,myth,bow,lassic,humboldt,accidentally,ishe,sail,suruc,reveng,trial,dk,ls,register,fuckboy,debris,unbelievably,tos,nowplay,slowpoke,dyersburg,id,Real_Target
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1
1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,4,1
2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,5,1
3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,6,1
4,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,7,1


In [ ]:
Final_tf_idf[l]

,gfe,american,deluxe,telerketing,elk,newbeginningsanilrescue,aicbomb,reflection,ph,metal,outrage,faroe,burger,delay,gosh,hardy,prison,wifi,discuss,gic,villa,gas,temecula,new,wolf,horizon,quarrel,plosive,promo,scar,os,cutest,coiff,crew,yzf,bordento,gal,ndle,captureyyc,beg,...,ferrell,ransomre,subtlety,highland,rds,bluio,notion,hamptonroadsfor,pronounce,engag,omron,service,faq,joseph,throwback,board,dealbreaker,pande,senator,plot,myth,bow,lassic,humboldt,accidentally,ishe,sail,suruc,reveng,trial,dk,ls,register,fuckboy,debris,unbelievably,tos,nowplay,slowpoke,dyersburg
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7357,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.470499,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
7358,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
7359,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
7360,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


## 2.2. Tokenización y Lemmatización (nltk)

In [32]:
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('punkt')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


True

In [33]:
# Tokenizar
tokenizer = nltk.RegexpTokenizer(r'\w+')
word_tok_nltk_en = [tokenizer.tokenize(doc) for doc in corpus_train]

In [34]:
word_tok_nltk_en[0] 

['for',
 'the',
 'people',
 'who',
 'di',
 'in',
 'hun',
 'periments',
 'by',
 'unit',
 'of',
 'japanese',
 'military']

In [43]:
type(corpus_train)

list

In [49]:
# Eliminar stop words
diccionarioT_nltk = set()
nltk_stop_words_en = set(nltk.corpus.stopwords.words("english"))
print(nltk_stop_words_en)

for doc in corpus_train:
  tokens = nltk.word_tokenize(doc)
  for t in tokens:
    if t not in nltk_stop_words_en:
      diccionarioT_nltk.add(t)

len(diccionarioT_nltk), diccionarioT_nltk

{'further', 'those', "don't", 'yourself', "hasn't", 'of', 'should', 'aren', 'their', "hadn't", 'if', "you're", "you've", 'other', 'each', 'only', 'y', 'himself', 'hers', 'having', 'does', "didn't", 'am', "shouldn't", "she's", 'doing', 'by', 'once', 'this', 'did', 'there', 'its', 'under', 'yours', 'while', 'how', 'ours', 'with', "weren't", 'is', 'into', 'them', 'a', 'off', 'd', 'or', 'at', 'during', 'wouldn', 'here', 'has', 'then', 'such', 'myself', 'own', 'him', 'and', 'mustn', 'over', 'so', 'she', 'after', 'ma', "isn't", 'hasn', 'don', 'shouldn', 'you', 'too', "couldn't", 'who', 'where', "you'd", 'isn', 'most', "mightn't", 'her', 'no', 'theirs', 'an', 'but', "needn't", "haven't", 'ain', "shan't", 'above', 'he', 'shan', 's', 'ourselves', 'be', 'was', 've', 'through', 'will', 'were', 'they', "should've", 'had', 'not', 'both', 'about', 'o', 'again', 'couldn', 'didn', 'his', 'yourselves', 'before', 'doesn', 'are', 'mightn', 're', 'it', 'any', 'nor', 'until', 'for', 'than', 'from', 'do', '

(10548,
 {'brits',
  'rketing',
  'infinity',
  'cack',
  'mgtab',
  'periments',
  'fees',
  'glanders',
  'done',
  'reebok',
  'stlo',
  'faces',
  'duction',
  'gladbach',
  'najibrazak',
  'auth',
  'ige',
  'son',
  'amid',
  'aggressively',
  'tottarian',
  'detectors',
  'plague',
  'clots',
  'yield',
  'lightn',
  'windows',
  'drob',
  'comics',
  'intrigu',
  'wwsiren',
  'allegations',
  'condemnation',
  'poverty',
  'boundary',
  'pand',
  'outbreaks',
  'rains',
  'championship',
  'spoke',
  'postering',
  'nycapricon',
  'seof',
  'pepper',
  'melines',
  'teang',
  'floral',
  'variety',
  'ally',
  'threatintel',
  'points',
  'concept',
  'drool',
  'seemeth',
  'weaknesses',
  'nevstis',
  'particulate',
  'wroug',
  'legend',
  'intain',
  'annoying',
  'crits',
  'giant',
  'spring',
  'looks',
  'repair',
  'unto',
  'welcomes',
  'ena',
  'churll',
  'stages',
  'surges',
  'thingsihate',
  'drinkers',
  'frightening',
  'moonlight',
  'denial',
  'wha',
  'ni

In [53]:
# Stemming
diccionarioS_nltk = set()
p_stemmer = nltk.stem.porter.PorterStemmer()
nltk_stemedList_en = [[p_stemmer.stem(word) for word in doc] for doc in word_tok_nltk_en_sw]

for doc in corpus_train:
  tokens = nltk.word_tokenize(doc)
  for t in tokens:
    t2 = p_stemmer.stem(t)
    if t2 not in nltk_stop_words_en:
      diccionarioS_nltk.add(t2)


print(len(diccionarioS_nltk))

8578


In [57]:
# Lemma
diccionarioL_nltk = set()
wordnet_lemmatizer = nltk.stem.WordNetLemmatizer()

for doc in corpus_train:
  tokens = nltk.word_tokenize(doc)
  for t in tokens:
    t2 = wordnet_lemmatizer.lemmatize(t)
    if t2 not in nltk_stop_words_en:
      diccionarioL_nltk.add(t2)

print(len(diccionarioL_nltk))

9556


# Modelo

In [36]:
X_train, y_train = Final_tf_idf_Train[tokens_Train] , Final_tf_idf_Train[["Real_Target"]]

In [37]:
X_test, y_test = Final_tf_idf_Test[tokens_Test] , Final_tf_idf_Test[["Real_Target"]]

In [ ]:
pipe = Pipeline([('clf', SVC(random_state=1234)),])
# tuneo 
#C, kernel y decision function shape
param_grid = dict(clf__C=[0.1],
                  clf__gamma = ['scale'],
                  clf__class_weight=['balanced'],
                  clf__decision_function_shape=['ovo'])

In [44]:
grid_search = GridSearchCV(pipe, param_grid=param_grid,cv=5,verbose=3,scoring='accuracy')

# Se ajusta el modelo con las diferentes configuraciones
grid_search.fit(X_train,y_train)

print(grid_search.best_params_)

Fitting 5 folds for each of 1 candidates, totalling 5 fits


/usr/local/lib/python3.7/dist-packages/sklearn/utils/validation.py:985: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


[CV 1/5] END clf__C=0.1, clf__class_weight=balanced, clf__decision_function_shape=ovo, clf__gamma=scale;, score=0.654 total time= 4.2min


/usr/local/lib/python3.7/dist-packages/sklearn/utils/validation.py:985: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


[CV 2/5] END clf__C=0.1, clf__class_weight=balanced, clf__decision_function_shape=ovo, clf__gamma=scale;, score=0.658 total time= 4.1min


...

[CV 4/5] END clf__C=0.1, clf__class_weight=balanced, clf__decision_function_shape=ovo, clf__gamma=scale;, score=0.635 total time= 4.1min


/usr/local/lib/python3.7/dist-packages/sklearn/utils/validation.py:985: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


[CV 5/5] END clf__C=0.1, clf__class_weight=balanced, clf__decision_function_shape=ovo, clf__gamma=scale;, score=0.640 total time= 4.1min


/usr/local/lib/python3.7/dist-packages/sklearn/utils/validation.py:985: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


{'clf__C': 0.1, 'clf__class_weight': 'balanced', 'clf__decision_function_shape': 'ovo', 'clf__gamma': 'scale'}


In [40]:
X_test.fillna(0,inplace=True), X_train.fillna(0,inplace=True)

/usr/local/lib/python3.7/dist-packages/pandas/core/frame.py:4327: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  downcast=downcast,


(None, None)

In [41]:
X_train.shape, y_train.shape, X_test.shape, y_test.shape

((5886, 9231), (5886, 1), (1501, 4136), (1501, 1))

In [42]:
lista_nuevas = list(set(X_train.columns)-set(X_test.columns))
for c in lista_nuevas:
  X_test[c] = 0

lista_nuevas = list(set(X_test.columns)-set(X_train.columns))
for c in lista_nuevas:
  X_train[c] = 0

X_train.shape, y_train.shape, X_test.shape, y_test.shape

((5886, 10958), (5886, 1), (1501, 10958), (1501, 1))

In [43]:
X_train.to_csv('./results/X_train.csv',index=False)
y_train.to_csv('./results/y_train.csv',index=False)

X_test.to_csv('./results/X_test.csv',index=False)
y_test.to_csv('./results/y_test.csv',index=False)

In [96]:
X_test.columns, X_train.columns

(Index(['drive', 'magichairbump', 'lerga', 'ambleside', 'dance', 'uncontrolled',
        'kowe', 'fraction', 'dual', 'tribune',
        ...
        'gasparc', 'carryi', 'glean', 'pawsox', 'musicadvisory',
        'vancouverisland', 'palter', 'alves', 'mcv', 'bright'],
       dtype='object', length=10958),
 Index(['brooms', 'nope', 'fraction', 'dual', 'brown', 'motivation', 'kamon',
        'sensitive', 'river', 'fashion',
        ...
        'uncles', 'anchor', 'ramag', 'connecticut', 'canal', 'biblical', 'ma',
        'danger', 'summerfate', 'blacklivesmatter'],
       dtype='object', length=10958))

In [98]:
X_test = X_test[X_train.columns] 
X_test.columns

Index(['brooms', 'nope', 'fraction', 'dual', 'brown', 'motivation', 'kamon',
       'sensitive', 'river', 'fashion',
       ...
       'uncles', 'anchor', 'ramag', 'connecticut', 'canal', 'biblical', 'ma',
       'danger', 'summerfate', 'blacklivesmatter'],
      dtype='object', length=10958)

In [100]:
rbf = svm.SVC(kernel = 'rbf',C= 0.1, decision_function_shape ='ovo', gamma = 'scale', class_weight= 'balanced',  random_state=1234)
rbf.fit(X_train, y_train)

accuracy_rbf = rbf.score(X_test, y_test)
print(accuracy_rbf)

y_pred = rbf.predict(X_test)
confusion_matrix = confusion_matrix(y_test, y_pred)

/usr/local/lib/python3.7/dist-packages/sklearn/utils/validation.py:985: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


0.6222518321119254


In [101]:
confusion_matrix

array([[439, 435],
       [132, 495]])

In [102]:
print('----------Model report on all classes ----------')
print(classification_report(y_test,y_pred))

----------Model report on all classes ----------
              precision    recall  f1-score   support

           0       0.77      0.50      0.61       874
           1       0.53      0.79      0.64       627

    accuracy                           0.62      1501
   macro avg       0.65      0.65      0.62      1501
weighted avg       0.67      0.62      0.62      1501

